<a href="https://colab.research.google.com/github/ParvaShah/Template_Extraction_NLP/blob/master/NLP_PROJECT_TASK_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**IMPORT LIBRARIES**

In [0]:
!pip install allennlp==1.0.0rc1 allennlp-models==1.0.0rc1
!pip install spacy==2.2.0

In [0]:
from google.colab import files
print('pred')
from allennlp.predictors.predictor import Predictor
print('coref')
import allennlp_models.coref
print('ner')
import allennlp_models.ner
print('consti')
import allennlp_models.syntax.constituency_parser
print('srl')
import allennlp_models.syntax.srl
print('/srl')
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import os
import math
import csv
import itertools
import pandas as pd
import re
import json
print('sp')
nlp = spacy.load("en_core_web_sm")
print('ner')
predictor_ner = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-model-2020.02.10.tar.gz")
print('coref')
predictor_coref = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")
print('consto')
predictor_constituency = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/elmo-constituency-parser-2020.02.10.tar.gz")
print('srl')
predictor_srl = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz")
print('import done')

In [0]:
predictor_ner._dataset_reader._token_indexers['token_characters']._min_padding_length = 3

#**COREFERENCE RESOLUTION**

In [0]:
def get_batched_corpus(dir_path,filename):

    record = 0
    sp_ch = 0
    corpus = []
    batch = []
    all_pr = []
    spcy_pr = []
    pronouns = ['all', 'another', 'any', 'anybody', 'anyone', 'anything', 'both', 'each', 'each', 'other', 'either', 'everybody', 'everyone', 'everything', 'few', 'he', 'her', 'hers', 'herself', 'him', 'himself', 'his', 'I', 'it', 'its', 'itself', 'little', 'many', 'me', 'mine', 'more', 'most', 'much', 'my', 'myself', 'neither', 'no one', 'nobody', 'none', 'nothing', 'one another', 'other', 'others', 'our', 'ours', 'ourselves', 'several', 'she', 'some', 'somebody', 'someone', 'something', 'that', 'their', 'theirs', 'them', 'themselves', 'these', 'they', 'this', 'those', 'us', 'we', 'what', 'whatever', 'which', 'whichever', 'who', 'whoever', 'whom', 'whomever', 'whose', 'you', 'your', 'yours', 'yourself', 'yourselves']
    pronouns = set([x.lower() for x in pronouns])
    with open(dir_path+filename, "r",encoding='UTF-8',errors='ignore') as f:
        content = f.readlines()
    ch = 0
    batch_pointer = 0
    for sentences in content:
        sentences = sentences.strip() + ' '
        if record < ch:
            record = ch
        length = len(sentences)
        if ch + length < 2000:
            batch.append(sentences)
            ch += length
            batch_pointer = 1
            continue
        wd_not_ch = sentences.split()
        for words in wd_not_ch:
            words = words.lower()
            pron = []
            for prons in pronouns:
                if prons == words:
                    pron.append(prons)
        all_pr.append(pron)
        if pron and ch + length < 3500 :
            batch.append(sentences)
            ch += length
            batch_pointer = 1
        else:
            corpus.append(batch)
            batch = []
            batch.append(sentences)
            ch = length
            batch_pointer = 0
    if batch_pointer == 1:
        corpus.append(batch)
    return corpus    

In [0]:
def merge_intervals(intervals):
    intervals.sort(key=lambda x: x[0])
    merged = []
    for interval in intervals:
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval)
        else:
            merged[-1][1] = max(merged[-1][1], interval[1])
    return merged

In [0]:
def do_coreference(coreferences,wds):
    #coreferences = model_coref['clusters']
    #wds = model_coref['document']
    wds_inx = [x for x in range(len(wds))]
    #input_sent = model_coref['document']
    wds_res = []
    wds_dict = {}
    ct = -1
    final_result = []
    for i in range(len(wds_inx)):
        wds_dict[i] = wds[i]
   
    for clusters in coreferences:
        for words in clusters:
            if wds[words[1]] == '\n':
                words[1] = words[1] - 1 
            if wds[words[0]] == '\n':
                words[0] = words[0] + 1

    #print('???????????????????????DICTIONARY',wds_dict)
    for clusters in coreferences:
        ner_word = ''
        #print(clusters)
        #print(' '.join(wds[clusters[0][0]:clusters[0][1]+1]))
        length = float('inf')
        final_head = (' '.join(wds[clusters[0][0]:clusters[0][1]+1])).strip()
        for i in range(len(clusters)):
            ner_word = (' '.join(wds[clusters[i][0]:clusters[i][1]+1])).strip()
            doc = nlp(ner_word)
            pred = [X.label_ for X in doc.ents]
            
            if pred:
                #print('compete:',ner_word)
                if len(ner_word) < length:
                    length = len(ner_word)
                    final_head = ner_word
        #print('final_head: ',final_head)
        wds_dict[ct]  = final_head
        for refs in clusters:
            for i in range(refs[0],refs[1]+1):
                wds_inx[i] = ct
        ct -= 1
    rec_ct = 0
    temp_wds = []
    for i in range(len(wds_inx)-1):
        #print(wds_inx[i],wds_inx[i+1])
        if wds_inx[i] == wds_inx[i+1]:
            temp_wds.append(i)
            rec_ct += 1
            continue
        if rec_ct > 4:
            for its in temp_wds:
                wds_res.append(its)
            wds_res.append(i)
            rec_ct = 0
            temp_wds = []
            continue

        wds_res.append(wds_inx[i])
        #print('append done')
        rec_ct = 0
        temp_wds = []

    if wds_inx[-1] != wds_inx[-2]:
        wds_res.append(wds_inx[-1])
    # print('IN MAIN DO COREF ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^6')
    # print(wds_dict)
    # print(wds_res)
    result = ''
    for ids in wds_res:
        word = wds_dict[ids]
        # print('word',word)
        if word == '\n':
            print(result)
            final_result.append(result.strip()+' ')
            result = ''
        result +=  word + ' '
    return final_result

In [0]:
def get_Coref(batch,original_file,coreference_file):

    coref_input = ''
    original_batch = []
    #print('Checking Original files!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    for i,sentence in enumerate(batch):
        #print(sentence)
        original_batch.append(sentence.strip())
        coref_input += sentence.strip()+'\n'
    model_coref = predictor_coref.predict(document=coref_input)   
    clusters = model_coref['clusters']
    words_coref = model_coref['document']
    # print(words_coref)
    word_range_dict = {}
    word_start = 0
    word_end = 0
    line_number = 0
    for words in words_coref:
        if words != '\n':
            word_end += 1
            continue
        else:
            word_range_dict[line_number] = [word_start,word_end-1]
            line_number += 1
            word_start = word_end + 1
            word_end += 1
    # print(word_range_dict)
    # print('Clustures:',clusters)
    merge_list = []
    for cluster in clusters:
        cluster_head = -1
        line_number = -1
        head_word = ''
        for word in cluster:
            start = word[0]
            end = word[1]
            for key,items in word_range_dict.items():
                if end <= items[1] and start >= items[0]:
                    line_number = key
            ner_word = ' '.join(words_coref[word[0]:word[1]+1])
            #print(ner_word)
            doc = nlp(ner_word)
            pred = [X.label_ for X in doc.ents if X.label_ == 'GPE' or  X.label_ == 'LOC' or X.label_ == 'PERSON'\
                    or  X.label_ == 'NORP' or  X.label_ == 'ORG' or  X.label_ == 'PRODUCT'or  X.label_ == 'EVENT'\
                    or X.label_ == 'LANGUAGE' or X.label_ == 'DATE']
            
            if len(ner_word) > 2 :
               
                model_ner = predictor_ner.predict(sentence=ner_word)
                for ner in (model_ner['tags']):
                    if len(ner) > 2:
                        pred.append(str(ner)[2:])
                        #print('ner_word####',ner_word)
            #print('word :',ner_word,pred)
            if pred or cluster_head == -1:
                head_word = ner_word
                cluster_head = line_number
            else:
                if cluster_head != line_number:
                    #print('########## word:',ner_word,line_number,head_word,cluster_head)
                    merge_list.append([cluster_head,line_number])
    #print(merge_list,'\n',len(merge_list))
    merge_list = merge_intervals(merge_list)
    #print('new',merge_list,'\n',len(merge_list))

        ### Merging list #####
    ### input: merge_list, clusters, batch, coref_input, words_coref
    coref_batch = do_coreference(clusters,words_coref)
    # print('Original Output:',original_batch)
    # print('Final_Output: ',coref_batch)
    # print('--------------------------------')
    # for i in range(len(coref_batch)):
    #     print('Coref: ',coref_batch[i])
    #     print('original: ',original_batch[i])

    # print('################################')
    # print("---------------------",len(original_batch),len(coref_batch),"--------------------")
    # if len(coref_batch) == len(original_batch):
    #     print("--------------------------------------------------------------True----------------------------------------")

    j = 0
    original_batch_merged = []
    coreference_batch_merged = []
    i = 0
    if merge_list:
        for sentence in (coref_batch):
            if j < len(merge_list):
                #print(merge_list[j])
                start = merge_list[j][0]
                end = merge_list[j][1]
                if start == i:
                    for k in range(start,end):
                        original_batch_merged.append(original_batch[k])
                        coreference_batch_merged.append(coref_batch[k])
                        # print(original_batch[k])
                        #if original_batch[k][-1] != '.':
                        original_batch_merged[len(original_batch_merged)-1] += '. '
                        coreference_batch_merged[len(coreference_batch_merged)-1] += '. '
                        #else:
                        #    original_batch_merged[len(original_batch_merged)-1] += ' '
                        #    coreference_batch_merged[len(coreference_batch_merged)-1] += ' '

                    j += 1
                    i += end - start
                    if i > len(coref_batch) - 1:
                        break
                original_batch_merged.append(original_batch[i]+'\n')
                coreference_batch_merged.append(coref_batch[i]+'\n')
                i += 1
    else:
        original_batch_merged = original_batch
        coreference_batch_merged = coref_batch

    # print(original_batch_merged)
    # print(coreference_batch_merged)
    # if len(original_batch_merged) == len(coreference_batch_merged):
    #     print("--------------------------------------------------------------True merged----------------------------------------")

    ######## Appending outputs to files ######

    og_file = open(original_file,"a+",encoding='UTF-8-sig')
    co_file = open(coreference_file,"a+",encoding='UTF-8-sig')
    length = len(coreference_batch_merged)
    for i,sentence in enumerate(coreference_batch_merged):
        og_file.write(original_batch_merged[i])
        co_file.write(coreference_batch_merged[i])

    og_file.close()
    co_file.close()

#**WORK TEMPLATE**

In [0]:
def get_job_list():
    rows = []
    with open('/content/drive/My Drive/job_title.csv', 'r') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # extracting each data row one by one 
        for row in csvreader: 
            rows.append(str(row).lower()[2:-2])
    rows = set(rows)
    #print(len(rows))
    rows_copy = rows.copy()
    for job in rows_copy:
        if len(job) < 3:
            rows.discard(job)
        
    #print(len(rows))
    #print(rows)
    position_list = rows
    position_list.add('board member')
    position_list.add('board of directors')
    return position_list

In [0]:
def getNPS(children,parent,dict_parent_child):
    
    if 'children' in children.keys():
        for sub_child in children['children']:
            if children['nodeType'] in set(['NP','PP','NX','S']): 
                if parent not in dict_parent_child.keys():
                    dict_parent_child[parent] = []
                if children['word'] not in set(dict_parent_child[parent]):
                    dict_parent_child[parent].append(children['word'])
            getNPS(sub_child,children['word'],dict_parent_child)

In [0]:
def is_Sublist(l, s):
  st = -1
  en = -1
  sub_set = False
  if s == []:
    return []
  elif s == l:
    return [0, len(s)-1]
  elif len(s) > len(l):
    sub_set = False

  else:
    for i in range(len(l)):
      if l[i] == s[0] and not sub_set:
        st = i
        n = 1
        while (n < len(s)) and (l[i+n] == s[n]):
          n += 1
        
        if n == len(s):
          en = i+n-1
          sub_set = True
  if st == -1 or en == -1:
    sub_set = False
    return []
  return [st, en]

In [0]:
def work_is_a(input_sentence,postion_list, original_words_list, original_tags_list):

    pred = predictor_constituency.predict(sentence=input_sentence)
    input_rec = pred['hierplane_tree']['root']
    dict_parent_child = {}
    
    getNPS(input_rec,'root',dict_parent_child)
    
    position_org_list = []
    for key, values in dict_parent_child.items():
        # key = key.replace(' - ','-')
        for vals in values:
            # vals = vals.replace(' - ','-')
            # vals = vals.replace('founderer','co-founder')
            position_org_list.append(vals)
            # print('CHILD: ',vals)
        # key = key.replace('founderer','co-founder')
        position_org_list.append(key)
        # print('PARENT: ',key)
    
    position_org_dict = {}
    location_org_dict = {}
    for line in position_org_list:
        sentences = line
        doc = nlp(line)
        # for token in doc:
        #     if token.dep_ != 'det':
        #         sentences += token.text + ' '
        # sentences = sentences.strip()
        # print("NP:- ",sentences)
        wds = sentences.split()
        if len(sentences) > 2 :
            # print("np  - - - - - - - - - -", line.split(' '))
            ## optimize here
            ## get model_ner['words'] and model_ner['tags']
            ## get start and end index '
            line_words_list = []
            line_tags_list = []
            list_index = is_Sublist(original_words_list, line.split(' '))
            if list_index:
                line_words_list = original_words_list[list_index[0]:list_index[1] + 1]
                line_tags_list = original_tags_list[list_index[0]:list_index[1] + 1]
            # model_ner = predictor_ner.predict(sentence=sentences)
            # print("final_word_list: - - - - - - - - - - - - - ", line_words_list)
            # print("final_tag_list: - - - - - - - - - - - - - ", line_tags_list)
            lt = ''
            organizations = []
            flag = 0
            combi_list =  ''
            for wd,ner in zip(line_words_list,line_tags_list):
                
                if wd == ',' or wd == 'and' or 'ORG' in ner:
                    flag = 1
                    if len(ner) > 2 and 'ORG' in ner:
                        if ner[0] == 'U':
                            combi_list += wd+', '
                            
                        else:
                            if ner[0] == 'L':
                                if lt:
                                    combi_list += lt+wd + ', '
                                    lt = ''
                            else:
                                lt += wd + ' '
                else:
                    flag = 0
                    if combi_list:
                        organizations.append(combi_list[:-2])
                        combi_list = ''
            if combi_list:
                organizations.append(combi_list[:-2])
            ############ LOC##########
            if not organizations:
                doc_org = nlp(sentences)
                for ent in doc_org.ents:
                    if ent.label_ == 'ORG':
                        organizations.append(ent.text)

            organizations = list(set(organizations))
            # print("organizations: ",organizations)
            if len(organizations) == 1:
                organizations = organizations[0]
                if organizations not in set(position_org_dict.keys()):
                    position_org_dict[organizations] = []
                    location_org_dict[organizations] = []
                
                doc1 = nlp(sentences)
                
                for chunk in doc1.noun_chunks:
                    ck = chunk.text
                    ck = ck.replace(' - ','-')
                    ck_list = ck.split('and')
                    
                    for small_chunks in ck_list:
                        # print('small_chunk', small_chunks)
                        small_chunks = small_chunks.strip()
                        if small_chunks == 'founderer':
                            small_chunks = 'co-founder'
                        if small_chunks.lower() in position_list:
                            
                            if small_chunks not in set(position_org_dict[organizations]):
                                position_org_dict[organizations].append(small_chunks)
                    
                    doc_loc = nlp(chunk.text)
                    for ent in doc_loc.ents:
                        
                        if ent.label_ == 'LOC' or ent.label_ == 'GPE':
                            
                            if ent.text not in set(location_org_dict[organizations]):
                                location_org_dict[organizations].append(ent.text)
                
    return position_org_dict,location_org_dict

In [0]:
def get_work_template(doc_file,position_list):
    ps_list_final = []
    lc_list_final = []
    person_list_final = []
    sent_final = []
    lc_template_final = []
    # print("Main line: ",doc_file)
    
    for sentence in doc_file.sents:
        # print("sentence: ", sentence)
        # print("-----------------------------")
        # continue
        ps_list = []
        lc_list = []
        person_list = []
        lc_template = []
        sent_temp = sentence
        sentence = str(sentence).strip()
        sentence = sentence.replace('co - founder','founderer')
        sentence = sentence.replace('board of directors','board member')
        sentence = sentence.replace('board of director','board member')
        doc_words = nlp(sentence)
        sentences = ''
        #print(sentence)
        for token in doc_words:
            if token.dep_ != 'det':
                sentences += token.text + ' '
        sentence = sentences.strip()
        ### Location Template
        lc_template = get_location_teplate(doc_words)
        # print('&*****&&&&&&&&*&&&&&&&&***********************',lc_template,'&*****&&&&&&&&*&&&&&&&&***********************')
        pred = [X.label_ for X in doc_words.ents]
        words = [X.text for X in doc_words.ents]
        wds = sentence.split()
        # print("pred: ",pred)
        # print("words: ",words)
        if len(sentence) > 2 :
            model_ner = predictor_ner.predict(sentence=sentence)
            # print("words_list - - - - - - - - - - - - - - - - - - - - - -  ", model_ner['words'])
            # print("tags_list - - - - - - - - - - - - - - - - - - - - - -  ", model_ner['tags'])
            lt = ''
            for ner,word in zip(model_ner['tags'], model_ner['words']):
                if len(ner) > 2 and 'PER' in ner:
                    if ner[0] == 'U':
                        person_list.append(word)
                        pred.append('PER')
                        
                    else:
                        if ner[0] == 'L':
                            person_list.append(lt+' '+word)
                            pred.append('PER')
                            lt = ''
                        else:
                            lt += word + ' '
        # print('person_list', person_list)
        if 'PERSON' in pred or 'PER' in pred:
            if 'ORG' in pred or 'NORP' in pred:
                position,location = work_is_a(sentence,position_list, model_ner['words'], model_ner['tags'])
                # print('---------################---------------')
                # print(position)
                # print('---------################---------------')
                ps_list_final.append(position)
                lc_list_final.append(location)
                person_list_final.append(person_list)
                sent_final.append(sentence)
        lc_template_final.append(lc_template)
    # print('&*****&&&&&&&&*&&&&&&&&***********************',lc_template,'&*****&&&&&&&&*&&&&&&&&***********************')
    return ps_list_final,lc_list_final,person_list_final,sent_final,lc_template_final

#**LOCATION TEMPLATE**

In [0]:
def get_location_list():
    places = []
    with open('/content/drive/My Drive/city_state_country.txt', 'r') as filehandle:
        for line in filehandle:
            currentPlace = line[:-1]
            currentPlace = currentPlace.strip('][').split(', ')
            currentPlace = [loc.strip("''") for loc in currentPlace ]
            places.append(currentPlace)
    
    preprocess_df = pd.read_csv("/content/drive/My Drive/city_state_country_preprocess.csv",header=None)
    codes = preprocess_df[0]
    vals = preprocess_df[1]
    preprocess_dict = {}

    for key, vals in zip(codes,vals):
        vals = vals.strip('][').split(', ')
        vals = [loc.strip("''") for loc in vals ]
        preprocess_dict[key] = vals
    preprocess_dict['uk'].append('united kingdom')
    return preprocess_dict,places

In [0]:
def get_location_teplate(doc):
    
    preprocess_dict,places = get_location_list()
    test = [str(X.text).lower() for X in doc.ents if X.label_ == 'GPE' or  X.label_ == 'LOC' ]
    final_keep = [str(X.text) for X in doc.ents if X.label_ == 'GPE' or  X.label_ == 'LOC' ]
    book_keep = {}
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    punctuations = punctuations+ '"'
    main_test = []
    for items in test:
        no_punct = ""
        for char in items:
            if char not in punctuations:
                no_punct = no_punct + char
        
        if no_punct in preprocess_dict:

            for abbrevations in preprocess_dict[no_punct]:
                
                main_test.append(''.join(abbrevations))
                
            if items not in book_keep:
                book_keep[items] = preprocess_dict[no_punct]
        else:
            main_test.append(no_punct)
    
    main_test = (list(itertools.combinations(main_test,2)))
    location_template = []
    for loc_1,loc_2 in main_test:
        for item in places:
            if [x for x in item if loc_1 == x] and [x for x in item if loc_2 == x]:
                if item.index(loc_1) > item.index(loc_2):
                    loc_1,loc_2 = loc_2,loc_1
                for key, val in book_keep.items():  
                    if loc_1 in val:
                        loc_1 = key
                    if loc_2 in val:
                        loc_2 = key
                l_1_inx = test.index(loc_1)
                l_2_inx = test.index(loc_2)
                loc_1 = final_keep[l_1_inx]
                loc_2 = final_keep[l_2_inx]
                if loc_1 == loc_2:
                    continue
                location_template.append([loc_1,loc_2])
    # print("template_loc:- ",location_template)
    return location_template    

#**BUY TEMPLATE**

In [0]:
def extract_buy_template(doc_input):

    final_list = []
    # print('################ In buy',doc_input)
    for sentence in doc_input.sents:
        input_sentence = str(sentence)
        pred = predictor_srl.predict(sentence = input_sentence)
        # print("---------------------------------")
        # print("sentence in Buy", input_sentence)
        buy_template_list = []
        # print('pred', pred['verbs'])
        for item in pred['verbs']:
            verb_doc = nlp(item['verb'])
            # print('verb:- ', verb_doc[0])
            if verb_doc[0].lemma_ in ['buy', 'acquire', 'purchase', 'get', 'score', 'obtain', 'sell']:
                verb_word = item['verb']
                tags = item['tags']
                # if verb_word in dict:
                arg0 = []
                arg1 = []
                arg2 = []
                other_args = []
                outside_words = []
                for tag,word in zip(tags, pred['words']):
                # print("tag:- ",tag," word:- ", word)
                    if tag[2:] == "V":
                        continue
                    if tag[2:] == "ARG0":
                        arg0.append(word)
                    elif tag[2:] == "ARG1":
                        arg1.append(word)
                    elif tag[2:] == "ARG2":
                        arg2.append(word)
                    elif tag == "O":
                        outside_words.append(word)
                    else:
                        other_args.append(word)
                # print("--------before-------")
                # print('args0:',arg0)
                # print('args1',arg1)
                # print('args2',arg2)
                # print('other',other_args)
                # print('outside',outside_words)
                # print('VERB:',verb_word)

                if verb_doc[0].lemma_ in ["sell"]:
                    arg0, arg2 = arg2, arg0
                    
                # print("--------After-------")
                # print('args0:',arg0)
                # print('args1',arg1)
                # print('args2',arg2)
                # print('other',other_args)
                # print('outside',outside_words)
            
                # create key : label dictionary
                
                model_ner = predictor_ner.predict(sentence=input_sentence)
                # print(model_ner)
                lt = ''
                keys = []
                labels = []
                for word,ner in zip(model_ner['words'],model_ner['tags']):
                    if len(ner) > 2:
                        if ner[0] == 'U':
                            if word not in keys:
                                keys.append(word)
                                labels.append(str(ner)[2:])
                            
                        else:
                            if ner[0] == 'L':
                                if lt:
                                    if (lt + word) not in keys:
                                        keys.append(lt + word)
                                        labels.append(str(ner)[2:])
                                    lt = ''
                            else:
                                lt += word + ' '
                # print('KEYS', keys)
                # print('LABELS', labels)
                
                buyer = ''
                item = ''
                price = ''
                quantity = ''
                source = ''
                # get buyer
                buyer = get_arg0(arg0, keys, labels)
                if buyer:
                    if not arg1 and not arg2:
                        # get item, source, price and quantity from other_args 
                        item, source, price, quantity = get_other_args(other_args, keys, labels, 1)
                    elif arg0 and arg1 and not arg2:
                        # get item from arg1
                        item, price, quantity = get_arg1(arg1, keys, labels)
                        # get source from other_args
                        xyz, source, price, quantity = get_other_args(other_args, keys, labels, 2)
                    elif arg0 and not arg1 and arg2:
                        # get source, price and quantity from arg2
                        source, price, quantity = get_arg2(arg2, keys, labels)
                        # get price, quantity from other_args
                        xyz, abc, price, quantity = get_other_args(other_args, keys, labels, 3)
                    else:
                        # get item from arg1
                        item, price, quantity = get_arg1(arg1, keys, labels)
                        # get source from arg2
                        source, price, quantity = get_arg2(arg2, keys, labels)
                        xyz, abc, price, quantity = get_other_args(other_args, keys, labels, 3)
                
                
                source_list = source.split(", ")
                # print("###########source:- ", source_list)
                for i in range(len(source_list)):
                    if source_list[i] in buyer or source_list[i] in item:
                        source_list[i] = ''
                source = ' '.join(source_list)
                final_list.append(buyer + "||" + item + "||" + price + "||" + quantity + "||" + source)
        #print("template:- ", buy_template_list)
        #final_list.append(buy_template_list)

    return final_list

In [0]:
def get_other_args(other_args, keys, labels, case):
    # case 1 : item - ORG, NORP, PRODUCT, Source - ORG, NORP, PER
    # case 2 : Source - ORG, NORP, PER
    # case 3 : MONEY, CARDINAL
    # print('other_args')
    args = ' '.join(other_args)
    item = ''
    source = ''
    price = ''
    quantity = ''
    # print(args)
    doc = nlp(args)
    for key in keys:
        #print(ent.text)
        if case == 1 :
            if key in args:
                if labels[keys.index(key)] in ['ORG','MISC']:
                    item = key + ', '
        if case == 2 :
            if key in args:
                if labels[keys.index(key)] in ['ORG', 'PER']:
                    source = key + ', '
        
        if labels[keys.index(key)] in ['CARDINAL']:
            #print(ent.text)
            quantity += key+', '
    for ent in doc.ents:
        # print("######## ent:- ",ent)
        if ent.label_ in ['MONEY']:
            #print(ent.text)
            price += ent.text+', '
    if not doc.ents:
        match = re.search(r'([A-Za-z]{0,3}?[\s]?[^\d\.\,\s][\s]?\d[\d,.]*(-\d[\d,.]*)?[\s]?[KMB]?)', args)
        if match:
            price += str(match.group()) + ', '
    item = item[:-2]
    source = source[:-2]
    price = price[:-2]
    quantity = quantity[:-2]
    # print('item: ', item)
    # print('source: ',source)
    # print('price',price)
    # print('qunt',quantity)
    return item, source, price, quantity

In [0]:
def get_arg2(arg2, keys, labels):
    ## arg2
    # print('arg2')
    args = ' '.join(arg2)
    source = ''
    price = ''
    quantity = ''
    #print(args)
    doc = nlp(args)
    for key in keys:
        #print(ent.text)
        if key in args:
            if labels[keys.index(key)] in ['ORG','PER']:
                #print(ent.text)
                source += key+', '
        if labels[keys.index(key)] in ['CARDINAL']:
            #print(ent.text)
            quantity += key+', '
    for ent in doc.ents:
        if ent.label_ in ['MONEY']:
            #print(ent.text)
            price += ent.text+', '
    if not doc.ents:
        match = re.search(r'([A-Za-z]{0,3}?[\s]?[^\d\.\,\s][\s]?\d[\d,.]*(-\d[\d,.]*)?[\s]?[KMB]?)', args)
        if match:
            price += str(match.group()) + ', '
    source = source[:-2]
    price = price[:-2]
    quantity = quantity[:-2]
    # print('source: ',source)
    # print('price',price)
    # print('qunt',quantity)
    return source, price, quantity

In [0]:
def get_arg1(arg1, keys, labels):
    ## arg1
    # print('arg1')
    args = ' '.join(arg1)
    item = ''
    price = ''
    quantity = ''
    #print(args)
    doc = nlp(args)
    for key in keys:
        if key in args:
            # print("####################In item arg1:- ",key)
            if labels[keys.index(key)] in ['ORG','MISC']:
                #print(ent.text)
                item += key + ', '
        if labels[keys.index(key)] in ['CARDINAL']:
            #print(ent.text)
            quantity += key + ', '
    for ent in doc.ents:
        if ent.label_ in ['MONEY']:
            #print(ent.text)
            price += ent.text+', '
    if not doc.ents:
        match = re.search(r'([A-Za-z]{0,3}?[\s]?[^\d\.\,\s][\s]?\d[\d,.]*(-\d[\d,.]*)?[\s]?[KMB]?)', args)
        if match:
            price += str(match.group()) + ', '
    item = item[:-2]
    price = price[:-2]
    quantity = quantity[:-2]
    # print('seller: ',seller)
    # print('amount',amount)
    # print('qunt',quantity)
    return item, price, quantity

In [0]:
def get_arg0(arg0, keys, labels):
    ## arg0
    # print('arg0')
    args = ' '.join(arg0)
    buyer = ''
    #print(args)
    # doc = nlp(args)
    for key in keys:
        #print(ent.text)
        if key in args:
            if labels[keys.index(key)] in ['PER','ORG']:
                #print(ent.text)
                buyer += key+', '
    buyer = buyer[:-2]
    # print('Buyer: ',buyer)
    return buyer

#**MAIN FUNCTION**

### **GET SEGMENTED FILES**

In [20]:
!mkdir 'segmented'
dir_path = '/content/drive/My Drive/WikipediaArticles/'
out_path = '/content/segmented/'
dir_in = '/content/segmented/'

for filename in os.listdir(dir_path):
    print(filename)
    myfile = open(dir_path+filename, "r",encoding='UTF-8',errors='ignore').read()
    doc_file = nlp(myfile)
    f = open(out_path+'segmented_'+filename,'w')
    for sentence in doc_file.sents:
        f.write(str(sentence).strip() + " ")
        f.write('\n')
    f.close()

1.txt
2.txt
3.txt
4.txt
5.txt


In [0]:
dir_path = '/content/segmented/'
out_path = '/content/segmented_1/'
!mkdir '/content/segmented_1/'
for filename in os.listdir(dir_path):
    with open(dir_path+filename, "r",encoding='UTF-8',errors='ignore') as file:
        content = file.readlines()
    f = open(out_path+filename,'w')
    for sentence in content:
        if len(sentence) > 1:
            f.write(str(sentence).strip() + ' ')
            f.write('\n')
    file.close()
    f.close()

###**GENERATED COREFERENCE AND ORIGINAL FILES**

In [30]:
dir_path = '/content/segmented_1/'

!mkdir original_files
!mkdir coreference_files
for filename in os.listdir(dir_path):
    filename = 'segmented_1.txt'
    original_file = '/content/original_files/'+filename
    coreference_file = '/content/coreference_files/'+filename
    corpus = get_batched_corpus(dir_path,filename)
    print(corpus)
    for batch in corpus:
        # print(batch)
        coreference_batch = get_Coref(batch,original_file,coreference_file)
    break    

mkdir: cannot create directory ‘original_files’: File exists
mkdir: cannot create directory ‘coreference_files’: File exists
[['In 2017, Amazon acquired Whole Foods Market for US$13.4 billion. ', 'Jeff Bezos is the CEO of Amazon. ', 'Amazon was founded by Jeff Bezos in Bellevue, Washington, in July 1994. ']]
In 2017 , Amazon acquired Whole Foods Market for US$ 13.4 billion . 

 Jeff Bezos is the CEO of Amazon . 

 Amazon was founded by Jeff Bezos in Bellevue , Washington , in July 1994 . 


###**EXTRACTING TEMPLATES**

In [0]:
######### work and location template ###########
file_coref = 'coreference_files/segmented_1.txt'
file_ori = 'original_files/segmented_1.txt'
final_result = []
reader_coref = open(file_coref,'r',encoding = 'utf-8',errors='ignore')
reader_ori = open(file_ori,'r',encoding = 'utf-8',errors='ignore')
position_list = get_job_list()
for line_coref,line_ori in zip(reader_coref,reader_ori):
    # print(line_coref)
    ## Work template
    #line_coref = "On October 3, 2017, reports indicated Powell Jobs had purchased stake in the ownership group Monumental Sports & Entertainment that includes the NBA's Washington Wizards, NHL's Washington Capitals, and Capital One Arena. Her approximately twenty percent stake makes her the second largest shareholder behind chairman Ted Leonsis."
    line_coref = line_coref.replace('\ufeff','')
    line_ori = line_ori.replace('\ufeff','')
    line_ori = line_ori.strip()
    doc_file_coref = nlp(line_coref)
    ### Working on segmented sentences of 1 coref sentence and 1 original sentence
    position_coref,location_coref,person_list,sent,loc_template =  get_work_template(doc_file_coref,position_list)
    # print('&*****&&&&&&&&*&&&&&&&&***********************',loc_template,'))))')
    ## removing duplicates
    # person list
    person_list_non_dups = []
    for item in person_list:
        if item:
            person_list_non_dups.append(list(set(item)))
    #position list
    position_coref_non_null = []
    for dd in position_coref:
        position_coref_non_null.append({key:val for key, val in dd.items() if val})
    
    #Location list
    location_coref_non_null = []
    for dd in location_coref:
        location_coref_non_null.append({key:val for key, val in dd.items() if val})
    # print('----------------BEFORE FINAL',loc_template)
    loc_template_non_null = [] 
    #list(k for k,_ in itertools.groupby(loc_template))
    for locations in loc_template:
        if locations:
            for lcs in locations:
                if lcs:
                    loc_template_non_null.append(lcs)

    loc_template_non_null.sort()
    loc_template_non_null = list(k for k,_ in itertools.groupby(loc_template_non_null))
    # print('IN LOCATION TEMPLATE',loc_template_non_null)
    ######## Work Template ##########################
    work_template_dict = []
    for persons, positions, locs in zip(person_list_non_dups,position_coref_non_null,location_coref_non_null):
        for person in persons:
            pos_dict = {}
            tmp_dict = {}
            if person:
                pos_dict['1'] = person
                for org, position in positions.items():
                    if position:
                        pos_dict['2'] = org
                        pos_dict['3'] = position
                        if org in locs.keys():
                            pos_dict['4'] = locs[org]
                        tmp_dict['sentences'] = line_ori
                        tmp_dict['template'] = 'WORK'
                        pos_tmp_dict = pos_dict.copy()
                        tmp_dict['arguments'] = pos_tmp_dict
                        temp = tmp_dict.copy()
                        work_template_dict.append(temp)
    # print('WORK: ',work_template_dict)
    
    ################# Location Template #######################
    location_template_dict = []
    for location in loc_template_non_null:
        loc_dict = {}
        tmp_dict = {}
        loc_dict['1'] = location[0]
        loc_dict['2'] = location[1]
        tmp_dict['sentence'] = line_ori
        loc_tmp_dict = loc_dict.copy()
        tmp_dict['argument'] = loc_tmp_dict
        tmp_dict['template'] = 'PART'
        temp = tmp_dict.copy()
        location_template_dict.append(temp)
    # print('######### LT: ',location_template_dict)
     
    ################# Buy Template ############################
    list_buy = extract_buy_template(doc_file_coref) 
    # print(list_buy)
    buy_dict = []
    for items in list_buy:
        buy = items.split('||')
        # print(buy)
        if buy[0]:
            tmp_dict = {}
            buy_sell_dict = {}
            buy_sell_dict['buyer'] = buy[0]
            buy_sell_dict['item'] = buy[1]
            buy_sell_dict['price'] = buy[2]
            buy_sell_dict['quantity'] = buy[3]
            buy_sell_dict['source'] = buy[4]
            tmp_dict['sentence'] = line_ori
            tmp_dict['template'] = 'BUY'
            buy_sell_tmp_dict = buy_sell_dict.copy()
            tmp_dict['arguments'] = buy_sell_tmp_dict
            temp = tmp_dict.copy()
            buy_dict.append(temp)
    # print('buy_template',buy_dict)
    if work_template_dict:
        final_result.append(work_template_dict)
    if location_template_dict:
        final_result.append(location_template_dict)
    if buy_dict:
        final_result.append(buy_dict)

json_string = json.dumps(final_result)
with open('result.json','w',encoding='utf-8') as file:
    json.dump(final_result,file,ensure_ascii=False,indent=2)        

In [32]:
parsed = json.loads(json_string)
print(json.dumps(parsed, indent=4, sort_keys=True))

[
    [
        {
            "arguments": {
                "1": "Jeff  Bezos",
                "2": "Amazon",
                "3": [
                    "CEO"
                ]
            },
            "sentences": "In 2017, Amazon acquired Whole Foods Market for US$13.4 billion.Jeff Bezos is the CEO of Amazon.Amazon was founded by Jeff Bezos in Bellevue, Washington, in July 1994.",
            "template": "WORK"
        }
    ],
    [
        {
            "argument": {
                "1": "Bellevue",
                "2": "Washington"
            },
            "sentence": "In 2017, Amazon acquired Whole Foods Market for US$13.4 billion.Jeff Bezos is the CEO of Amazon.Amazon was founded by Jeff Bezos in Bellevue, Washington, in July 1994.",
            "template": "PART"
        }
    ],
    [
        {
            "arguments": {
                "buyer": "Amazon",
                "item": "Whole Foods Market",
                "price": "US$ 13.4 billion",
                "quantity"